In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras import layers

Using TensorFlow backend.


In [2]:
def Generator():
    generator_input = keras.Input(shape=(256, 256, 1))
    
    x = layers.Conv2D(16, kernel_size=4, strides=2, padding='same')(generator_input)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(32, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(1, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    

    x = layers.Dense(8*16*16*128)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((8, 16, 16, 128))(x)

    x = layers.Conv3DTranspose(64, kernel_size=4, strides=2 , padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv3DTranspose(32, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv3DTranspose(16, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv3DTranspose(1, kernel_size=4, strides=(1,2,2), activation='tanh', padding='same')(x)

    generator = keras.models.Model(generator_input, x)
    print(generator.summary())
    
    return generator

In [3]:
def Discriminator():
    discriminator_input = keras.Input(shape=(64, 256, 256, 1))

    y = layers.Conv3D(16, kernel_size=4, strides=2, padding='same')(discriminator_input)
    y = layers.LeakyReLU()(y)

    y = layers.Conv3D(32, kernel_size=4, strides=2, padding='same')(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Conv3D(64, kernel_size=4, strides=2, padding='same')(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Conv3D(1, kernel_size=4, strides=2, padding='same')(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Flatten()(y)
    y = layers.Dropout(0.4)(y)
    y = layers.Dense(1, activation="sigmoid")(y)

    discriminator = keras.models.Model(discriminator_input, y)
    print(discriminator.summary())
    
    return discriminator    

In [4]:
generator = Generator()
discriminator = Discriminator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      272       
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 16)      64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        8224      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 32)        0         
__________

In [5]:
# Defun gives 10 secs/epoch performance boost
generator.call = tf.contrib.eager.defun(generator.call)
discriminator.call = tf.contrib.eager.defun(discriminator.call)

In [6]:
def discriminator_loss(real_output, generated_output):
    # [1,1,...,1] with real output since it is true and we want
    # our generated examples to look like it
    real_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.ones_like(real_output), logits=real_output)

    # [0,0,...,0] with generated images since they are fake
    generated_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.zeros_like(generated_output), logits=generated_output)

    total_loss = real_loss + generated_loss

    return total_loss

In [7]:
def generator_loss(input_frames, generated_output):
    distance_L1 = tf.reduce_sum(tf.abs(tf.subtract(input_frames, generated_output[:, 0])))
    return tf.losses.sigmoid_cross_entropy(tf.ones_like(generated_output), generated_output) + (distance_L1*0.0008)

In [8]:
discriminator_optimizer = tf.train.AdamOptimizer(1e-4)
generator_optimizer = tf.train.AdamOptimizer(1e-4)

In [9]:
import matplotlib.pyplot as plt

def display_first_frames(first_frames):

    fig = plt.figure(figsize=(256,256))
  
    for i in range(20):
        plt.subplot(256, 256, i+1)
        print(type(first_frames[i].eval(session=sess)))
        plt.imshow(np.asarray(first_frames[i] * 127.5 + 127.5, dtype='float32'), cmap='gray')
        plt.axis('off')

In [15]:
%run input_pipeline.ipynb

In [17]:
import time
import os
import skvideo.io

save_dir = '/../../../../newvolume/gen_videos'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
for epoch in range(NUM_EPOCHS):

    batch_videos = sess.run(next_batch)
    gen_input = get_batch_of_initial_frames(batch_videos)
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_videos = generator.predict(gen_input)

        real_output = discriminator.train_on_batch(batch_videos)
        generated_output = discriminator.train_on_batch(generated_videos)

        gen_loss = generator_loss(gen_input, generated_output)
        disc_loss = discriminator_loss(real_output, generated_output)    

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.variables))

    if epoch % 100 == 0:
        discriminator.save_weights('discriminator_3.h5')
        generator.save_weights('generator_3.h5')
        print(epoch)
        print('discriminator loss:', sess.run(disc_loss))
        print('adversarial loss:', sess.run(gen_loss))
        np_gen_videos = np.asarray(generated_videos.eval())
        #         display_first_frames(tf.unstack(generated_videos[:, 0]))
        show_sample(np_gen_videos[0][0].reshape(256, 256), np_gen_videos[10][16].reshape(256, 256))
        for i in range(20):
            video = ((np_gen_videos[i] - np_gen_videos[i].min()) * (1/(np_gen_videos[i].max() - np_gen_videos[i].min()) * 255)).astype('uint8')
            skvideo.io.vwrite(os.path.join(save_dir, "generated_video_{}.mp4".format(i)), video)    

ValueError: If your data is in the form of symbolic tensors, you should specify the `steps` argument (instead of the `batch_size` argument, because symbolic tensors are expected to produce batches of input data).